In [ ]:
pip install --upgrade vllm

In [2]:
import importlib.metadata
importlib.metadata.version("torch")

'2.1.2'

In [3]:
importlib.metadata.version("vllm")

'0.4.0.post1'

In [4]:
importlib.metadata.version("xformers")

'0.0.23.post1'

In [5]:
importlib.metadata.version("flash_attn")

'2.5.6'

In [1]:
model = "solidrust/Nous-Hermes-2-Mistral-7B-DPO-AWQ"
max_model_len=32000

In [2]:
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model)

# Pass the default decoding hyperparameters of Qwen1.5-7B-Chat
# max_tokens is for the maximum length for generation.
sampling_params = SamplingParams(temperature=0.7, top_p=0.8, repetition_penalty=1.05, max_tokens=512)

llm = LLM(model, quantization="awq", kv_cache_dtype="fp8_e5m2", gpu_memory_utilization=0.99)
#llm = LLM(model, quantization="gptq", kv_cache_dtype="fp8_e5m2", max_model_len=max_model_len)
#llm = LLM(model, quantization="gptq", max_model_len=max_model_len)
#llm = LLM(model, quantization="awq", max_model_len=max_model_len)
#llm = LLM(model, quantization="awq", kv_cache_dtype="fp8_e5m2", max_model_len=max_model_len)

/workspace/wordslab-llms/.venv/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WARNING 04-07 08:51:57 config.py:211] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 04-07 08:51:57 utils.py:256] Not found nvcc in /usr/local/cuda. Skip cuda version check!
INFO 04-07 08:51:57 config.py:381] Using fp8_e5m2 data type to store kv cache. It reduces the GPU memory footprint and boosts the performance. But it may cause slight accuracy drop. Currently we only support fp8 without scaling factors and make e5m2 as a default format.
INFO 04-07 08:51:57 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='solidrust/Nous-Hermes-2-Mistral-7B-DPO-AWQ', tokenizer='solidrust/Nous-Hermes-2-Mistral-7B-DPO-AWQ', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=awq, enforce_eager=False, kv_cache_dtype=fp8_e5m2, device_config=

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WARNING 04-07 08:51:58 utils.py:357] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.
INFO 04-07 08:51:58 selector.py:16] Using FlashAttention backend.
INFO 04-07 08:51:59 weight_utils.py:177] Using model weights format ['*.safetensors']
INFO 04-07 08:52:00 model_runner.py:104] Loading model weights took 3.8823 GB
INFO 04-07 08:52:04 gpu_executor.py:94] # GPU blocks: 15949, # CPU blocks: 4096
INFO 04-07 08:52:04 model_runner.py:791] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-07 08:52:04 model_runner.py:795] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 04-07 08:52:08 model_runner.py:867] Graph capturin

In [3]:
import time

In [4]:
prompt = "Quels sont les avantages du Crédit Mutuel ?"

# System prompt
messages = [
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": prompt}
]

# Generate outputs
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
start_time = time.time()  # Record the start time
outputs = llm.generate([text], sampling_params)
end_time = time.time()  # Record the end time
    
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}, Performance: {int(len(output.outputs[0].token_ids)/(end_time-start_time))} tokens/sec")

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.69s/it]

Prompt: '<|im_start|>system\nTu es un assistant utile et professionnel qui répond toujours en français.<|im_end|>\n<|im_start|>user\nQuels sont les avantages du Crédit Mutuel ?<|im_end|>\n<|im_start|>assistant\n', Generated text: "Le Crédit Mutuel est un groupe bancaire coopératif et mutuel qui offre plusieurs avantages à ses adhérents :\n\n1. Solidarité financière : Le Crédit Mutuel est basé sur la solidarité financière, c'est-à-dire que les bénéfices réalisés par les établissements du groupe sont redistribués entre les adhérents sous forme de remises d'intérêt ou de dividendes.\n\n2. Tarifs compétitifs : Le Crédit Mutuel propose généralement des tarifs compétitifs sur ses produits bancaires, notamment pour le prêt, le crédit et les cartes bancaires.\n\n3. Service client : Les établissements du Crédit Mutuel sont réputés pour leur service client de qualité, avec un accompagnement personnelisé et une attention portée aux besoins individuels des adhérents.\n\n4. Stabilité financière : E

- model = "Qwen/Qwen1.5-32B-Chat-GPTQ-Int4"
- max_model_len=10576
- quantization="gptq"
- kv_cache_dtype="fp8_e5m2"
- gpu_memory_utilization=0.99

42 tokens / sec

- model = "Qwen/Qwen1.5-14B-Chat-GPTQ-Int4"
- quantization="gptq"
- kv_cache_dtype="fp8_e5m2"
- max_model_len=22624

80 tokens/sec

- model = "Qwen/Qwen1.5-14B-Chat-GPTQ-Int4"
- quantization="gptq"
- max_model_len=11744

80 tokens/sec

- model = "Qwen/Qwen1.5-14B-Chat-GPTQ-Int8"
- quantization="gptq"
- kv_cache_dtype="fp8_e5m2"
- max_model_len=4016
- gpu_memory_utilization=1

50 tokens/sec

- model = "Qwen/Qwen1.5-14B-Chat-AWQ"
- quantization="awq"
- max_model_len=10544

74 tokens/sec

- model = "Qwen/Qwen1.5-14B-Chat-AWQ"
- quantization="awq"
- kv_cache_dtype="fp8_e5m2"
- max_model_len=21104

74 tokens/sec



Tongyi Qianwen LICENSE AGREEMENT

- "We"(or "Us") shall mean Alibaba Cloud.
- You are granted a non-exclusive, worldwide, non-transferable and royalty-free limited license under Alibaba Cloud's intellectual property or other rights owned by Us embodied in the Materials to use, reproduce, distribute, copy, create derivative works of, and make modifications to the Materials.
- Restrictions: If you are commercially using the Materials, and your product or service has more than 100 million monthly active users, You shall request a license from Us. You cannot exercise your rights under this Agreement without our express authorization.
- Rules of use: **You can not use the Materials or any output therefrom to improve any other large language model (excluding Tongyi Qianwen or derivative works thereof).**
- Governing Law and Jurisdiction: This Agreement and any dispute arising out of or relating to it will be governed by the laws of China

- model = "NousResearch/Hermes-2-Pro-Mistral-7B"
- max_model_len=12544
- gpu_memory_utilization=0.95

55 tokens/sec

- solidrust/Nous-Hermes-2-Mistral-7B-DPO-AWQ
- quantization="awq"
- kv_cache_dtype="fp8_e5m2"
- gpu_memory_utilization=0.99
- (max_seq_len=32768)

125 tokens/sec

Hermes 2 Pro - Mistral 7B LICENSE : Apache 2.0